In [17]:
from paho.mqtt import client as mqtt
import sqlite3
import json
from datetime import datetime

In [36]:
broker = 'localhost'
port = 8000
topic1 = "location0/data"
topic2 = "location1/data"
topic3 = "location2/data"
client_id = f'python-mqtt-{10}'

In [37]:
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(topic1)
    client.subscribe(topic2)
    client.subscribe(topic3)


mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message
mqttc.connect(broker, port)
mqttc.loop_forever()

Connected with result code Success
location1/data
location0/data
location2/data


KeyboardInterrupt: 

In [40]:
sqliteConnection = sqlite3.connect('database.db')

In [32]:
# change the on_message function such that msg.payload json is parsed and added to the database
def on_message(client, userdata, msg):
    data = json.loads(msg.payload)
    print(msg.topic)
    cursor = sqliteConnection.cursor()
    cursor.execute("INSERT INTO sensor_data (timestamp, node, temperature, humidity) VALUES (?,?,?,?)", (data['timestamp'], msg.topic ,data['temperature'], data['humidity']))
    sqliteConnection.commit()
    cursor.close()

In [29]:
# create the correct table
cursor = sqliteConnection.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS sensor_data (id INTEGER PRIMARY KEY AUTOINCREMENT, node STRING , timestamp DATETIME, temperature FLOAT, humidity FLOAT)")
sqliteConnection.commit()
cursor.close()

In [21]:
datetime.now()

datetime.datetime(2024, 4, 17, 21, 49, 54, 161903)

In [41]:
# show the data in the database
cursor = sqliteConnection.cursor()
cursor.execute("SELECT * FROM sensor_data")
rows = cursor.fetchall()
cursor.close()

In [42]:
rows

[(1, 'location2/data', 'Wed Apr 17 22:49:32 2024', 43.0, 2222.0),
 (2, 'location0/data', 'Wed Apr 17 22:49:32 2024', 43.0, 2222.0),
 (3, 'location0/data', 'Wed Apr 17 22:49:32 2024', 43.0, 2222.0),
 (4, 'location1/data', 'Wed Apr 17 22:49:32 2024', 43.0, 2222.0),
 (5, 'location2/data', 'Wed Apr 17 22:49:32 2024', 43.0, 2222.0)]